In [1]:
from nba_api.stats.endpoints import shotchartdetail, playercareerstats
from nba_api.stats.static import players, teams
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class NbaScraper:
    """ Class to scrape data from the NBA official website.
    """
    @staticmethod
    def get_json_from_name(name: str, is_player=True) -> int:
        """ Get the json of a player or team from his name
        """
        from nba_api.stats.static import players, teams
        if is_player:
            nba_players = players.get_players()
            return [player for player in nba_players 
                    if player['full_name'] == name][0]
        else:
            nba_teams = teams.get_teams()
            return [team for team in nba_teams 
                    if team['full_name'] == name][0]
    
    @staticmethod
    def get_player_career(player_id: int) -> list:
        """ Get the career of a player from his id
        """
        from nba_api.stats.endpoints import playercareerstats
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        return career.get_data_frames()[0]
    
    @staticmethod
    def get_shot_data(id: int, team_ids: list, seasons: list) -> list:
        """ Get the shot data of a player from his id and seasons
        """
        from nba_api.stats.endpoints import shotchartdetail
        df = pd.DataFrame()
        for season in seasons:
            for team in team_ids:
                shot_data = shotchartdetail.ShotChartDetail(
                    team_id=team,
                    player_id=id,
                    context_measure_simple='FGA',
                    season_nullable=season
                )
                df = pd.concat([df, shot_data.get_data_frames()[0]])
        
        return df
    
    @staticmethod
    def get_all_ids(only_active=True) -> list:
        """ Get all the ids of the players
        """
        from nba_api.stats.static import players
        nba_players = players.get_players()
        if only_active:
            return [player['id'] for player in nba_players 
                    if player['is_active']]
        return [player['id'] for player in nba_players]
    
    @staticmethod
    def get_player_headshot(id: int) -> str:
            """ Get the headshot of a player from his id
            """
            from nba_api.stats.static import players
            import requests
            import shutil
            
            url = f'https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/{id}.png'
            output_path = f'../data/nba/transient/headshots/{id}.png'
            r = requests.get(url, stream=True)
            if r.status_code == 200:
                with open(output_path, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
    
    @staticmethod                                    
    def get_all_nba_headshots(only_active=False) -> None:
        """ Get the headshots of all the players
        """
        ids = NbaScraper.get_all_ids(only_active=only_active)
        for id in ids:
            NbaScraper.get_player_headshot(id)

### Ktaegorije: sezona, posamezna tekma, posamezna ekipa, čas tekme, pozicija meta

In [3]:
# get player id
name = "Luka Doncic"
id = NbaScraper.get_json_from_name(name)['id']
print(id)
career = NbaScraper.get_player_career(id)
teams = ['DAL']
team_ids = list(set(career[career.TEAM_ABBREVIATION.isin(teams)].TEAM_ID.values))

seasons = ['2018-19','2019-20','2020-21','2021-22','2022-23','2023-24','2024-25']
shot_data = NbaScraper.get_shot_data(id, team_ids, seasons)
print(len(shot_data))
shot_data.tail()

1629029
8597


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
296,Shot Chart Detail,0022400232,396,1629029,Luka Dončić,1610612742,Dallas Mavericks,3,8,35,...,Left Side Center(LC),24+ ft.,27,-157,223,1,0,20241116,DAL,SAS
297,Shot Chart Detail,0022400232,417,1629029,Luka Dončić,1610612742,Dallas Mavericks,3,7,32,...,Center(C),Less Than 8 ft.,1,5,17,1,1,20241116,DAL,SAS
298,Shot Chart Detail,0022400232,464,1629029,Luka Dončić,1610612742,Dallas Mavericks,3,5,11,...,Center(C),24+ ft.,25,-20,258,1,0,20241116,DAL,SAS
299,Shot Chart Detail,0022400232,482,1629029,Luka Dončić,1610612742,Dallas Mavericks,3,3,50,...,Center(C),Less Than 8 ft.,3,26,23,1,1,20241116,DAL,SAS
300,Shot Chart Detail,0022400232,561,1629029,Luka Dončić,1610612742,Dallas Mavericks,4,11,17,...,Center(C),24+ ft.,28,-45,285,1,0,20241116,DAL,SAS


In [4]:
# Assuming shot_data is a pandas DataFrame
shot_data.to_csv('../data/shot_data.csv', index=False)

In [5]:
game_dates = shot_data['GAME_DATE'].unique()
game_dates

array(['20181017', '20181020', '20181022', '20181024', '20181026',
       '20181028', '20181029', '20181031', '20181102', '20181106',
       '20181107', '20181110', '20181112', '20181114', '20181117',
       '20181119', '20181121', '20181124', '20181128', '20181130',
       '20181204', '20181205', '20181208', '20181210', '20181212',
       '20181213', '20181216', '20181218', '20181220', '20181222',
       '20181223', '20181226', '20181228', '20181230', '20181231',
       '20190102', '20190104', '20190105', '20190107', '20190109',
       '20190111', '20190113', '20190116', '20190119', '20190121',
       '20190122', '20190125', '20190127', '20190130', '20190202',
       '20190206', '20190208', '20190210', '20190211', '20190213',
       '20190225', '20190227', '20190302', '20190304', '20190306',
       '20190308', '20190310', '20190312', '20190314', '20190318',
       '20190320', '20190321', '20190323', '20190326', '20190328',
       '20190403', '20190409', '20191023', '20191025', '201910

In [6]:
# Extract unique GAME_DATE values
game_dates = shot_data['GAME_DATE'].unique()

# Convert GAME_DATE to JavaScript Date objects
js_dates = []
for date in game_dates:
    year = int(date[:4])
    month = int(date[4:6]) - 1  # JavaScript months are 0-indexed
    day = int(date[6:8])
    js_dates.append(f'new Date({year}, {month}, {day})')

# Generate the JavaScript code
js_code = 'var dates = [\n    ' + ',\n    '.join(js_dates) + '\n  ];'
print(js_code)

var dates = [
    new Date(2018, 9, 17),
    new Date(2018, 9, 20),
    new Date(2018, 9, 22),
    new Date(2018, 9, 24),
    new Date(2018, 9, 26),
    new Date(2018, 9, 28),
    new Date(2018, 9, 29),
    new Date(2018, 9, 31),
    new Date(2018, 10, 2),
    new Date(2018, 10, 6),
    new Date(2018, 10, 7),
    new Date(2018, 10, 10),
    new Date(2018, 10, 12),
    new Date(2018, 10, 14),
    new Date(2018, 10, 17),
    new Date(2018, 10, 19),
    new Date(2018, 10, 21),
    new Date(2018, 10, 24),
    new Date(2018, 10, 28),
    new Date(2018, 10, 30),
    new Date(2018, 11, 4),
    new Date(2018, 11, 5),
    new Date(2018, 11, 8),
    new Date(2018, 11, 10),
    new Date(2018, 11, 12),
    new Date(2018, 11, 13),
    new Date(2018, 11, 16),
    new Date(2018, 11, 18),
    new Date(2018, 11, 20),
    new Date(2018, 11, 22),
    new Date(2018, 11, 23),
    new Date(2018, 11, 26),
    new Date(2018, 11, 28),
    new Date(2018, 11, 30),
    new Date(2018, 11, 31),
    new Date(2019, 0